In [ ]:
import nest_asyncio
import asyncio
import aiohttp
import time

nest_asyncio.apply()

# API Details
URL = "https://core-api-green-bird-4902.fly.dev/analyze-complexity"
HEADERS = {"Content-Type": "application/json"}
PAYLOAD = {"text": "Aristotle was the smartest of his time"}

# Test parameters
NUM_REQUESTS = 20  # Total number of requests
CONCURRENCY = 5    # Max concurrent requests

async def send_request(session, semaphore):
    """Send a single request while respecting concurrency limits."""
    async with semaphore:
        start = time.perf_counter()
        try:
            async with session.post(URL, json=PAYLOAD, headers=HEADERS) as resp:
                await resp.text()  # Read response content
        except Exception as e:
            print(f"Request failed: {e}")
        end = time.perf_counter()
        return end - start

async def benchmark():
    """Run benchmark test with concurrent requests."""
    semaphore = asyncio.Semaphore(CONCURRENCY)
    async with aiohttp.ClientSession() as session:
        tasks = [send_request(session, semaphore) for _ in range(NUM_REQUESTS)]
        latencies = await asyncio.gather(*tasks)

    avg_latency = sum(latencies) / len(latencies)
    rps = len(latencies) / sum(latencies)

    print("\n=== Benchmark Results ===")
    print(f"Total Requests: {NUM_REQUESTS}")
    print(f"Concurrency Level: {CONCURRENCY}")
    print(f"Average Latency: {avg_latency*1000:.2f} ms")
    print(f"Requests Per Second (RPS): {rps:.2f}")

async def main():
    """Run benchmark test and measure total execution time."""
    start_time = time.perf_counter()
    await benchmark()
    total_time = time.perf_counter() - start_time

    print(f"Total Time Taken: {total_time:.2f}s")
    print(f"Average Requests per Second (RPS): {NUM_REQUESTS/total_time:.2f}")

# Run the test
asyncio.run(main())


In [ ]:

import nest_asyncio
import asyncio
import aiohttp
import time

nest_asyncio.apply()

# API Details
URL = "https://knowledge-graph-broken-surf-6085.fly.dev/extract-knowledge-graph"
HEADERS = {"Content-Type": "application/json"}
PAYLOAD = {"text": "Aristotle was the smartest of his time"}

# Test parameters
NUM_REQUESTS = 20  # Total number of requests
CONCURRENCY = 5    # Max concurrent requests

async def send_request(session, semaphore):
    """Send a single request while respecting concurrency limits."""
    async with semaphore:
        start = time.perf_counter()
        try:
            async with session.post(URL, json=PAYLOAD, headers=HEADERS) as resp:
                await resp.text()  # Read response content
        except Exception as e:
            print(f"Request failed: {e}")
        end = time.perf_counter()
        return end - start

async def benchmark():
    """Run benchmark test with concurrent requests."""
    semaphore = asyncio.Semaphore(CONCURRENCY)
    async with aiohttp.ClientSession() as session:
        tasks = [send_request(session, semaphore) for _ in range(NUM_REQUESTS)]
        latencies = await asyncio.gather(*tasks)

    avg_latency = sum(latencies) / len(latencies)
    rps = len(latencies) / sum(latencies)

    print("\n=== Benchmark Results ===")
    print(f"Total Requests: {NUM_REQUESTS}")
    print(f"Concurrency Level: {CONCURRENCY}")
    print(f"Average Latency: {avg_latency*1000:.2f} ms")
    print(f"Requests Per Second (RPS): {rps:.2f}")

async def main():
    """Run benchmark test and measure total execution time."""
    start_time = time.perf_counter()
    await benchmark()
    total_time = time.perf_counter() - start_time

    print(f"Total Time Taken: {total_time:.2f}s")
    print(f"Average Requests per Second (RPS): {NUM_REQUESTS/total_time:.2f}")

# Run the test
asyncio.run(main())


In [ ]:
core_url = "https://surbhitcodes--core-api-fastapi.modal.run"
kg_url = "https://surbhitcodes--knowledge-graph-api-fastapi.modal.run"
ta_url ="https://surbhitcodes--text-analysis-api-fastapi.modal.run"

In [ ]:
url="https://surbhitcodes--text-analysis-api-fastapi.modal.run/extract-analyze-complexity-stream"


In [3]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

core_url = "https://surbhitcodes--core-api-fastapi.modal.run/analyze-complexity"
kg_url = "https://surbhitcodes--knowledge-graph-api-fastapi.modal.run/extract-knowledge-graph"
ta_url = "https://surbhitcodes--text-analysis-api-fastapi.modal.run/extract-analyze-complexity-stream"


headers = {"Content-Type": "application/json"}
payload = {"text": "Aristotle was the smartest of his time"}
NUM_REQUESTS = 100

concurrent_requests = 0
peak_concurrency = 0
lock = threading.Lock()

def send_request(url):
    global concurrent_requests, peak_concurrency

    with lock:
        concurrent_requests += 1
        peak_concurrency = max(peak_concurrency, concurrent_requests)

    start_time = time.time()
    try:
        response = requests.post(url, json=payload, headers=headers, timeout=20)
        latency = (time.time() - start_time) 
        return response.status_code, latency, response.text[:200]  # Include error message
    except requests.exceptions.RequestException as e:
        return "ERROR", None, str(e)  # Capture error message
    finally:
        with lock:
            concurrent_requests -= 1

def test_api(url, api_name):
    global peak_concurrency
    peak_concurrency = 0

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=NUM_REQUESTS) as executor:
        futures = [executor.submit(send_request, url) for _ in range(NUM_REQUESTS)]
        results = [future.result() for future in as_completed(futures)]
    total_time = (time.time() - start_time)

    success_responses = [latency for status, latency, _ in results if status == 200]
    failed_responses = [(status, msg) for status, _, msg in results if status != 200]

    success_count = len(success_responses)
    error_count = len(failed_responses)
    avg_latency = round(sum(success_responses) / len(success_responses), 3) if success_responses else 0
    max_latency = round(max(success_responses), 3) if success_responses else 0
    min_latency = round(min(success_responses), 3) if success_responses else 0

    print(f"\n{api_name} API Load Test Results")
    print(f"Total Requests: {NUM_REQUESTS}")
    print(f"Successful Requests: {success_count}")
    print(f"Failed Requests: {error_count}")
    print(f"Peak Concurrent Requests: {peak_concurrency}")
    print(f"Total Test Duration: {round(total_time, 3)}s")
    print(f"Latency (Avg: {avg_latency}s, Min: {min_latency}s, Max: {max_latency}s)")

    if failed_responses:
        print("\n3 Sample Error Responses:")
        for status, msg in failed_responses[:3]:
            print(f"Status: {status}, Error: {msg}")

test_api(core_url, "Core")
# test_api(kg_url, "Knowledge Graph")
test_api(ta_url, "Text Analysis")



Core API Load Test Results
Total Requests: 100
Successful Requests: 100
Failed Requests: 0
Peak Concurrent Requests: 100
Total Test Duration: 13.482s
Latency (Avg: 4.371s, Min: 2.501s, Max: 13.438s)

Text Analysis API Load Test Results
Total Requests: 100
Successful Requests: 100
Failed Requests: 0
Peak Concurrent Requests: 100
Total Test Duration: 9.111s
Latency (Avg: 4.231s, Min: 2.483s, Max: 9.066s)
